In [1]:
# importing standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
import seaborn as sns

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet


from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

from itertools import combinations

import statsmodels.api as sm
import scipy.stats as stats

In [7]:
# importing data from .csv file
df = pd.read_csv('../data/analytic_data2019.csv')

In [8]:
df

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,...,Male population 18-44 raw value,Male population 45-64 raw value,Male population 65+ raw value,Total male population raw value,Female population 0-17 raw value,Female population 18-44 raw value,Female population 45-64 raw value,Female population 65+ raw value,Total female population raw value,Population growth raw value
0,statecode,countycode,fipscode,state,county,year,county_ranked,v001_rawvalue,v001_numerator,v001_denominator,...,v013_rawvalue,v016_rawvalue,v017_rawvalue,v025_rawvalue,v026_rawvalue,v027_rawvalue,v031_rawvalue,v032_rawvalue,v035_rawvalue,v097_rawvalue
1,00,000,00000,US,United States,2019,NaN,6900.630354,3745538,908266078,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,000,01000,AL,Alabama,2019,NaN,9917.2328984,80440,13636816,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,001,01001,AL,Autauga County,2019,1,8824.0571232,815,156132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,003,01003,AL,Baldwin County,2019,1,7224.6321603,2827,576496,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56,37,56037,WY,Sweetwater County,2019,1,7497.44,495,127427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3191,56,39,56039,WY,Teton County,2019,1,3786.13,124,66351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3192,56,41,56041,WY,Uinta County,2019,1,7790.3,262,59466,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3193,56,43,56043,WY,Washakie County,2019,1,5504.65,108,22335,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# keeping first 6 columns and rawvalues
columns_to_drop = []
for column in df.columns[6:]:
    if not (df[column][0]).endswith('rawvalue'):
        columns_to_drop.append(column)
        
df.drop(columns_to_drop, inplace = True, axis = 1)

In [10]:
# dropping first row, which is the second header
df.drop(0, axis = 0, inplace = True)

In [11]:
# dropping aggregate rows
df.shape

(3194, 111)

In [12]:
df = df[df['County FIPS Code'] != '000']

In [13]:
df = df[df['County FIPS Code'] != 0]

In [14]:
df.shape

(3142, 111)

In [15]:
# dropping columns missing more than 10% of the data
columns_to_drop = []
for column in df.columns:
    if df[column].isna().sum() > len(df) / 10:
        columns_to_drop.append(column)
        
df.drop(columns_to_drop, axis = 1, inplace = True)

In [16]:
len(columns_to_drop)

43

In [18]:
# checking for remaining missing data
df_missing_values = pd.DataFrame()

missing_number = []
column_name = []

for column in df.columns:
    column_name.append(column)
    missing_number.append(df[column].isna().sum())
    
df_missing_values['column_name'] = column_name
df_missing_values['missing_number'] = missing_number

In [19]:
df_missing_values = df_missing_values.sort_values('missing_number', ascending = False)

In [20]:
df_missing_values.head(10)

,column_name,missing_number
23,Mental health providers raw value,252
34,Violent crime raw value,191
18,Sexually transmitted infections raw value,158
19,Teen births raw value,144
21,Primary care physicians raw value,135
53,Children eligible for free or reduced price lu...,121
10,Low birthweight raw value,107
35,Injury deaths raw value,103
27,High school graduation raw value,99
22,Dentists raw value,85


In [29]:
# filling remaining missing values with median value 
for column in df.columns[6:]:
    df[column] = df[column].map(lambda x: float(x))

In [30]:
state_median_df = df.groupby('State Abbreviation').median()

In [31]:
state_median_df.head()

,Premature death raw value,Poor or fair health raw value,Poor physical health days raw value,Poor mental health days raw value,Low birthweight raw value,Adult smoking raw value,Adult obesity raw value,Food environment index raw value,Physical inactivity raw value,Access to exercise opportunities raw value,...,% 65 and older raw value,% Non-Hispanic African American raw value,% American Indian and Alaskan Native raw value,% Asian raw value,% Native Hawaiian/Other Pacific Islander raw value,% Hispanic raw value,% Non-Hispanic white raw value,% not proficient in English raw value,% Females raw value,% Rural raw value
State Abbreviation,,,,,,,,,,,,,,,,,,,,,
AK,7876.536065,0.154541,3.745194,3.442669,0.058552,0.187008,0.3220,6.90,0.211,0.786260,...,0.125880,0.009324,0.163031,0.024990,0.003460,0.049265,0.498513,0.009650,0.472936,0.793375
AL,10875.865439,0.222160,4.699272,4.740404,0.101167,0.207356,0.3710,7.10,0.309,0.483473,...,0.184502,0.230050,0.005291,0.005651,0.000738,0.026710,0.689275,0.005943,0.514528,0.712322
AR,10744.462101,0.226897,4.893776,4.968565,0.084203,0.207739,0.3580,6.60,0.345,0.532240,...,0.190218,0.081519,0.007149,0.005667,0.000569,0.035295,0.796092,0.005745,0.507171,0.650608
AZ,7833.688943,0.208533,4.509992,4.033836,0.070935,0.156047,0.2970,6.20,0.243,0.666617,...,0.183112,0.012587,0.043069,0.011259,0.001833,0.300955,0.541257,0.038441,0.499798,0.331972
CA,6435.574344,0.163038,3.685637,3.820639,0.064516,0.123397,0.2445,7.85,0.182,0.870061,...,0.153316,0.018185,0.024039,0.044984,0.003119,0.256328,0.518193,0.065316,0.501381,0.160382


In [32]:
overall_median_df = df.median()

In [33]:
overall_median_df.head()

State FIPS Code                 29.000000
County FIPS Code                79.000000
5-digit FIPS Code            29176.000000
Release Year                  2019.000000
Premature death raw value     8130.554003
dtype: float64

In [36]:
def impute_per_state_median(row, column):
    """
    Checks if a value in a row is missing. 
    If it is missing, first tries to replace it with the median of the values derived from the same state. 
    If all the values are missing for a certain state (for example, Hawaii - water pollution), 
    replaces it with the median of the whole column. 
    
    Parameters
    ----------
    row : current row in the dataframe we want to populate
    
    column: the value that we want to populate
    
    
    Returns
    ----------
    row[column] if it's not missing, otherwise the median of state if not missing, 
    otherwise the median of the whole country
    
    """
    
    if math.isnan(row[column]):
        current_state = row['State Abbreviation']
        current_state_median = state_median_df.loc[current_state][column]
        overall_median = overall_median_df[column]
        if math.isnan(current_state_median):
            return overall_median
        else:
            return current_state_median
    else:
        return row[column]

In [37]:
# code runs for some time
for column in df.columns[6:]:
    df[column] = df.apply(lambda x: impute_per_state_median(x, column), axis = 1)

In [38]:
# checking if we are still missing values
df.isna().sum().sort_values()

State FIPS Code                                 0
Air pollution - particulate matter raw value    0
Drinking water violations raw value             0
Severe housing problems raw value               0
Driving alone to work raw value                 0
                                               ..
Uninsured raw value                             0
Teen births raw value                           0
Sexually transmitted infections raw value       0
Preventable hospital stays raw value            0
% Rural raw value                               0
Length: 68, dtype: int64

In [40]:
df.to_csv('../data/data_cleaned.csv', index = False)